In [3]:
import pandas as pd 
import numpy as np
import re
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
import pickle 
from theme_index import *

# 종목별 주가 import

In [4]:
# 주가 데이터 불러오기
stock_price = pd.read_csv('./data/230929_price_list.csv',header=None)

In [5]:
# 데이터 컬럼 정리
stock_price2 = stock_price.drop(stock_price.columns[0], axis=1)
stock_price2.columns = ['일자'] + stock_price2.iloc[1][1:].tolist()
stock_price2 = stock_price2.drop(index=[0,1])
stock_price2['일자'] = pd.to_datetime(stock_price2['일자'])

In [6]:
stock_price2

,일자,동화약품,KR모터스,경방,삼양홀딩스,하이트진로,유한양행,CJ대한통운,하이트진로홀딩스,두산,...,윙입푸드,SBI핀테크솔루션즈,엑세스바이오,잉글우드랩,코오롱티슈진,JTC,미투젠,소마젠,프레스티지바이오파마,네오이뮨텍
2,2023-02-22,9120,513,10930,75100,24400,51300,83400,10140,85600,...,1018,3975,10700,4840,9590,4515,12780,7300,10010,3755
3,2023-02-23,9170,512,10970,78500,24850,51400,83800,10220,91100,...,1017,4030,11020,4840,9610,4490,12810,7250,10110,3745
4,2023-02-24,9090,516,10940,78000,24850,53500,83300,10130,90400,...,1012,4235,10790,4775,9560,4490,12600,7040,10170,3615
5,2023-02-27,9090,503,10850,78300,24100,52800,82500,9990,100000,...,1001,4370,10380,4700,9430,4540,12420,6780,10500,3570
6,2023-02-28,9020,512,10980,78800,23900,52700,81700,9940,101000,...,1000,4375,10130,4860,9450,4530,12740,6810,10460,3585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,2023-09-21,9800,728,8790,69500,19680,73200,81100,8620,115000,...,922,5000,12110,12100,8770,3905,11100,5700,11150,1676
148,2023-09-22,9730,743,8720,70000,19320,73500,81500,8550,114600,...,905,4995,12200,11900,8800,3765,10840,5720,11210,1628
149,2023-09-25,9620,715,8770,69800,19290,74100,79800,8580,109500,...,898,4840,11450,12260,8680,3765,10830,5580,11010,1591
150,2023-09-26,9580,731,8540,69800,18860,74300,80600,8480,111500,...,897,4925,11450,11550,8660,3785,10580,5660,10820,1571


# 기업 공시자료 import

In [7]:
# 2023-01-03 ~ 2023-08-07 까지의 기업 공시자료
dart = pd.read_csv("./data/document.csv")
dart.head()

,Unnamed: 0,corp_code,corp_name,stock_code,report_nm,report
0,0,302926,현대로템,64350,분기보고서 (2023.03),II. 사업의 내용</TITLE>\n<P></P>\n\n<LIBRARY>\n<SEC...
1,0,201450,SK렌터카,68400,분기보고서 (2023.03),II. 사업의 내용</TITLE>\n\n<LIBRARY>\n<SECTION-2 AC...
2,0,411808,제이오,418550,[기재정정]사업보고서 (2022.12),"II. 사업의 내용</TITLE>\n<P USERMARK=""F-GL""></P>\n\..."
3,0,1442966,마스턴프리미어리츠,357430,사업보고서 (2023.03),II. 사업의 내용</TITLE>\n<P></P>\n\n<LIBRARY>\n<SEC...
4,0,167280,폴루스바이오팜,7630,분기보고서 (2022.09),II. 사업의 내용</TITLE>\n<P></P>\n\n<LIBRARY>\n<SEC...


In [8]:
dart2 = dart.copy()
# 상장 폐지 종목 제거
dart2 = dart2[dart2.corp_name.isin(stock_price2.columns)]
# 기업공시자료에서 report 전처리
dart2['report'] = dart2['report'].apply(dart_preprocess)

In [9]:
# 보고서 일자 추출
date_pattern = r'\d{4}\.\d{2}'
dart2['date'] = dart2['report_nm'].apply(lambda text: re.search(date_pattern, text).group() if re.search(date_pattern, text) else None)

# 기업별 최신 데이터만 남기기 
dart2['date'] = pd.to_datetime(dart2['date'])
dart2 = dart2.sort_values(by = ['corp_name','date'], ascending = False).groupby('corp_name').first().reset_index()

<ipython-input-9-b7fc523818fb>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dart2['date'] = pd.to_datetime(dart2['date'])


In [10]:
# IR 자료 불러오기
IR = pd.read_excel("./data/IR.xlsx")
IR = IR[['기업명', '텍스트']]
IR.columns = ['corp_name', 'ir_text']

In [11]:
IR = IR.drop_duplicates(subset=['corp_name'], keep='first')

In [12]:
dart2 = pd.merge(dart2, IR, on='corp_name', how='left')
# nan 값을 빈 문자열로 대체
dart2['ir_text'].fillna('', inplace=True)
dart2['report'] = dart2['report'] + dart2['ir_text']

del dart2['ir_text']

# 테마,키워드 import

In [13]:
# 6월 1주차 테마, 키워드 데이터
themes1 = pd.read_excel("./data/news_theme_data_230603-230609_combined.xlsx")
# 6월 2주차 테마, 키워드 데이터
themes2 = pd.read_excel('./data/news_theme_data_230805-230811_combined.xlsx')
# 6월 3주차 테마, 키워드 데이터
themes3 = pd.read_excel("./data/news_theme_data_230617-230623_combined.xlsx")


In [14]:
themes_lite1, co_related_keywords1 = theme_preprocessing(themes1)
themes_lite2, co_related_keywords2 = theme_preprocessing(themes2)
themes_lite3, co_related_keywords3 = theme_preprocessing(themes3)

# KR-SBERT 각 기업별 테마 유사도 계산 

In [15]:
# 사전 학습 임베딩 모델
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 

In [16]:
# 1~3주차별 공시자료-테마 문서 유사도 행렬 산출
#co_imb_similarity1 = imbedding_sim(theme_key_doc=True, theme_key_dic=co_related_keywords1, document=dart2, model = model)
co_imb_similarity2 = imbedding_sim(theme_key_doc=True, theme_key_dic=co_related_keywords2, document=dart2, model = model)
#co_imb_similarity3 = imbedding_sim(theme_key_doc=True, theme_key_dic=co_related_keywords3, document=dart2, model = model)

# 20분 정도 소요됩니다.

100%|██████████| 74/74 [00:03<00:00, 18.72it/s]


In [14]:
# 유사도 행렬 저장
co_imb_similarity1.to_csv('data/co_imb_similarity1.csv')
co_imb_similarity2.to_csv('data/co_imb_similarity2.csv')
co_imb_similarity3.to_csv('data/co_imb_similarity3.csv')

# 네이버 테마별 종목과 유사도 비교

In [15]:
# 위 코드를 다 돌리지 않았을 시에, 아래 코드의 주석 처리를 해제하고 데이터 불러올 수 있음.
#co_imb_similarity1 = pd.read_csv('./data/co_imb_similarity1.csv').set_index('corp_name')
#co_imb_similarity2 = pd.read_csv('./data/co_imb_similarity2.csv').set_index('corp_name')
#co_imb_similarity3 = pd.read_csv('./data/co_imb_similarity3.csv').set_index('corp_name')

# 1~3주차별 테마 종목 추출
co_imb_themed_stock1 = themed_stock(co_imb_similarity1, 0.4, 20)
co_imb_themed_stock2 = themed_stock(co_imb_similarity2, 0.4, 20)
co_imb_themed_stock3 = themed_stock(co_imb_similarity3, 0.4, 20)



100%|██████████| 148/148 [00:00<00:00, 4130.26it/s]


In [16]:

# 네이버 테마별 종목 데이터
with open('./data/naver_themed_stock.pickle', 'rb') as fr:
    naver_themed_stock = pickle.load(fr)

In [17]:
# 1~3주차별 테마 종목 정확도 계산
co_imb_accuracy1 = naver_accuracy(co_imb_themed_stock1, naver_themed_stock)
co_imb_accuracy2 = naver_accuracy(co_imb_themed_stock2, naver_themed_stock)
co_imb_accuracy3 = naver_accuracy(co_imb_themed_stock3, naver_themed_stock)

In [18]:
# 1~3주차별 테마 종목 정확도
print(np.array(list(co_imb_accuracy1.values())).mean())
print(np.array(list(co_imb_accuracy2.values())).mean())
print(np.array(list(co_imb_accuracy3.values())).mean())

0.3181818181818182
0.3286666666666667
0.2979375


# 테마지수 계산

In [19]:
# 위에서 주가 데이터를 불러오지 않았을 경우를 대비해 다시 주가 데이터를 불러옵니다.

# 주가 데이터 불러오기
stock_price = pd.read_csv('./data/230807_price_list.csv',header=None)
# 데이터 컬럼 정리
stock_price2 = stock_price.drop(stock_price.columns[0], axis=1)
stock_price2.columns = ['일자'] + stock_price2.iloc[1][1:].tolist()
stock_price2 = stock_price2.drop(index=[0,1])
stock_price2['일자'] = pd.to_datetime(stock_price2['일자'])

# 테마별 종목 정보, 종목별 주가 정보를 이용하여 테마지수 계산
theme_index1 = theme_index(co_imb_themed_stock1, stock_price2, '2023-05-22', '2023-06-09')
theme_index2 = theme_index(co_imb_themed_stock2, stock_price2, '2023-05-29', '2023-06-16')
theme_index3 = theme_index(co_imb_themed_stock3, stock_price2, '2023-06-05', '2023-06-23')

c:\Users\SUIN\OneDrive\SUIN\vscode\news\final_code\theme_index.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  theme_index[theme] = list(temp.mean(axis=1))
c:\Users\SUIN\OneDrive\SUIN\vscode\news\final_code\theme_index.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  theme_index[theme] = list(temp.mean(axis=1))
c:\Users\SUIN\OneDrive\SUIN\vscode\news\final_code\theme_index.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

In [20]:
# 테마 지수 데이터 저장
theme_index1.to_csv('data/theme_index_week1.csv')
theme_index2.to_csv('data/theme_index_week2.csv')
theme_index3.to_csv('data/theme_index_week3.csv')

In [ ]:
# 테마 종목 dictionary 저장
#with open('data/co_imb_themed_stock_week1.pickle','wb') as fw:
#    pickle.dump(co_imb_themed_stock1, fw)

#with open('data/co_imb_themed_stock_week2.pickle','wb') as fw:
#    pickle.dump(co_imb_themed_stock2, fw)
    
#with open('data/co_imb_themed_stock_week3.pickle','wb') as fw:
#    pickle.dump(co_imb_themed_stock3, fw)